In [2]:
using CarrierCapture
using Plots, LaTeXStrings
using DataFrames

# Potentials

In [ ]:
Qmin = -5
Qmax = 35
NQ = 3000
Q = range(Qmin, stop=Qmax, length=NQ)

In [ ]:
# pot1
nev = 30
name = "d0+e+h"
Q1 = [0.0000 0.8070 1.6142 2.4214 3.2286 4.0357 4.8428 5.6501 6.4571 7.2643 8.0715 8.8787 9.6857 10.4930 11.3000 12.1072 12.9144 13.7215 14.5287 15.3359 16.1430 16.9502 17.7574 18.5644 19.3716 20.1788 20.9859 21.7931 22.6002]
E1 = [-953.1843 -991.3164 -1019.6837 -1041.2625 -1058.1281 -1070.3129 -1078.6854 -1084.3352 -1088.1507 -1090.7382 -1092.4813 -1093.6187 -1094.3120 -1094.6724 -1094.7791 -1094.6931 -1094.4715 -1094.2920 -1094.1575 -1094.0447 -1093.9488 -1093.8619 -1093.7599 -1093.6182 -1093.4127 -1093.1290 -1092.7661 -1092.3560 -1091.9551]

pot1 = potential(); pot1.name = name
pot1.nev = nev
pot1.Q0 = Q1[findmin(E1)[2]]; pot1.E0 = 1.69834
pot1.QE_data = DataFrame(Q = Q1[:], E = E1[:])
pot1.QE_data.E .+= - minimum(pot1.QE_data.E) + pot1.E0
pot1.Q = Q

pot1.func_type = "bspline"
fit_pot!(pot1, Q)


# pot2
nev = 60
name = "DX-+h"
# spline fitting parameters
order = 4
smoothness = 0.001
weight = [1 1 1 1 1 0.5 0.4 0.4 0.5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]

Q2 = [30.66721918 29.860133 29.05306268 28.24612992 27.43911044 26.63197583 25.82507425 25.01797262 24.21096115 23.40387798 22.59690043 21.78991117 20.98281579 20.17570172 19.36884219 18.56169249 17.75463179 16.94772679 16.14061031 15.33347439 14.52663309 13.71945696 12.91240658 12.10544441 11.29841968 10.4913338 9.684370388 8.877289725 8.070184138]
E2 = [8.0902 7.5970 7.0749 6.5242 5.9461 5.3451 4.7300 4.1147 3.5182 2.9637 2.4769 2.0819 1.7972 1.6315 1.5800 1.6237 1.7301 1.8586 1.9687 2.0283 2.0190 2.0673 1.9910 2.0528 1.9730 2.0857 2.4550 3.1653 4.3448]

pot2 = potential(); pot2.name = name
pot2.nev = nev
pot2.Q0 = Q2[findmin(E1)[2]]; pot2.E0 = 1.69834
pot2.QE_data = DataFrame(Q = Q2[:], E = E2[:])
pot2.QE_data.E .+= - minimum(pot2.QE_data.E) + pot2.E0
pot2.Q = Q


pot2.func_type = "spline"
params = Dict()
params["order"] = order
params["smoothness"] = smoothness
params["weight"] = weight
fit_pot!(pot2, Q; params=params)


# pot3
nev = 110
name = "d0"

pot3 = potential(); pot3.name = name
pot3.nev = nev
pot3.Q0 = Q1[findmin(E1)[2]]; pot3.E0 = 0
pot3.QE_data = DataFrame(Q = Q1[:], E = E1[:])
pot3.QE_data.E .+= - minimum(pot3.QE_data.E) + pot3.E0
pot3.Q = Q

pot3.func_type = "bspline"
fit_pot!(pot3, Q; params=params)

solve_pot!(pot1)
solve_pot!(pot2)
solve_pot!(pot3)


### Capture barriers

In [ ]:
Qx, Ex = find_crossing(pot1, pot2)
println("Electron capture barrier: $(Ex-pot1.E0) eV")
Qx, Ex = find_crossing(pot2, pot3)
println("Hole capture barrier: $(Ex-pot2.E0) eV")

### Potentials and associated wave functions 

In [ ]:
plt = Plotter.plot_pot!(pot1; lplt_wf = false, color = "#b2182b")
Plotter.plot_pot!(pot2; lplt_wf = false, plt = plt, color = "#2166ac")
Plotter.plot_pot!(pot3; lplt_wf = false, plt = plt, color = "#b2182b")
xlims!((5, 25))
ylims!((-0.2, 3.7))

# Capture coefficient

In [ ]:
Cmin = 1E-20
Cmax = 1E-5
invTmin = 1
invTmax = 13
Tmin = 10
Tmax = 800
NT = 100
Volume = 4.72276E-21
cut_off = 0.25
σ = 0.01
temperature = range(Tmin, stop=Tmax, length=NT)

## Electron capture

In [ ]:
W = 0.0675 # e-ph coupling
g = 1 # degeneracy

# build a configuration coordinate for the electron capture
cc_e = conf_coord(pot1, pot2)
cc_e.W = W
cc_e.g = g
calc_overlap!(cc_e; cut_off = cut_off, σ = σ)
calc_capt_coeff!(cc_e, Volume, temperature)

## Hole capture

In [ ]:
W = 0.0217 # e-ph coupling
g = 1 # degeneracy

# build a configuration coordinate for the electron capture
cc_h = conf_coord(pot2, pot3)
cc_h.W = W
cc_h.g = g
calc_overlap!(cc_h; cut_off = cut_off, σ = σ)
calc_capt_coeff!(cc_h, Volume, temperature)

In [ ]:
plt = plot()
Plotter.plot_cc!(cc_e; plt=plt, label="electron capture")
Plotter.plot_cc!(cc_h; plt=plt, label="hole capture")

xlims!(1, 13)
ylims!(1E-29, 1E-7)